<a href="https://colab.research.google.com/github/caltunay/dd_competition_heart_disease/blob/master/retake_28_08_heart_diz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd 
import numpy as np

test_vals = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Heart Disease Prediction/test_values (1).csv'

train_vals = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Heart Disease Prediction/train_values (1).csv'
train_labs = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Heart Disease Prediction/train_labels (1).csv'

In [0]:
all = pd.merge(left = pd.read_csv(train_vals), right = pd.read_csv(train_labs), on = 'patient_id', how = 'inner')

In [0]:
cats = 'slope_of_peak_exercise_st_segment thal chest_pain_type num_major_vessels fasting_blood_sugar_gt_120_mg_per_dl resting_ekg_results sex exercise_induced_angina'.split()

cats

['slope_of_peak_exercise_st_segment',
 'thal',
 'chest_pain_type',
 'num_major_vessels',
 'fasting_blood_sugar_gt_120_mg_per_dl',
 'resting_ekg_results',
 'sex',
 'exercise_induced_angina']

In [0]:
cats.pop(cats.index('fasting_blood_sugar_gt_120_mg_per_dl'))
cats.pop(cats.index('exercise_induced_angina'))
cats.pop(cats.index('sex'))

cats

['slope_of_peak_exercise_st_segment',
 'thal',
 'chest_pain_type',
 'num_major_vessels',
 'resting_ekg_results']

In [0]:
ser_serum, bins_serum = pd.qcut(x = all['serum_cholesterol_mg_per_dl'], q = 3, retbins = True, labels = 'low average high'.split())
ser_serum.rename('serum_bin', inplace = True)

In [0]:
ser_hr, bins_hr = pd.qcut(x = all['max_heart_rate_achieved'], retbins = True, q = 4, labels = 'low med high too_high'.split())
ser_bp, bins_bp = pd.qcut(x = all['resting_blood_pressure'], retbins = True, q = 4, labels = 'low med high too_high'.split())

ser_hr.rename('heart_rate_bin', inplace = True)
ser_bp.rename('blood_pressure_bin', inplace = True)

print(bins_hr, bins_bp)

[ 96.   132.   152.   166.25 202.  ] [ 94. 120. 130. 140. 180.]


In [0]:
ser_age, bins_age = pd.qcut(x = all['age'], q = 2, retbins = True, labels = 'young old'.split())

ser_age.rename('age_bin', inplace = True)
# pd.concat([all, ser_age], axis = 1).groupby('age_bin').mean()['heart_disease_present']

In [0]:
ser_depr, bins_depr = pd.qcut(all['oldpeak_eq_st_depression'], q = 3, retbins = True, labels = 'low mid high'.split())

ser_depr.rename('depression_bin', inplace = True)

In [0]:
all = pd.concat([all, ser_serum, ser_hr, ser_bp, ser_age, ser_depr], axis = 1)
all.head()

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present,serum_bin,heart_rate_bin,blood_pressure_bin,age_bin,depression_bin
0,0z64un,1,normal,128,2,0,0,2,308,0.0,1,45,170,0,0,high,too_high,med,young,low
1,ryoo3j,2,normal,110,3,0,0,0,214,1.6,0,54,158,0,0,low,high,low,young,high
2,yt1s1x,1,normal,125,4,3,0,2,304,0.0,1,77,162,1,1,high,high,med,old,low
3,l2xjde,1,reversible_defect,152,4,0,0,0,223,0.0,1,40,181,0,1,low,too_high,too_high,young,low
4,oyt4ek,3,reversible_defect,178,1,0,0,2,270,4.2,1,59,145,0,0,high,med,too_high,old,high


In [0]:
all[cats].nunique()

slope_of_peak_exercise_st_segment    3
thal                                 3
chest_pain_type                      4
num_major_vessels                    4
resting_ekg_results                  3
dtype: int64

In [0]:
to_drop = 'serum_cholesterol_mg_per_dl max_heart_rate_achieved resting_blood_pressure age oldpeak_eq_st_depression'.split()

all = all.drop(labels = to_drop, axis = 1)

In [0]:
all_dummied = pd.get_dummies(data = all, columns = cats, prefix_sep = '|', drop_first = True)

all_dummied.head()

,patient_id,fasting_blood_sugar_gt_120_mg_per_dl,sex,exercise_induced_angina,heart_disease_present,serum_bin,heart_rate_bin,blood_pressure_bin,age_bin,depression_bin,slope_of_peak_exercise_st_segment|2,slope_of_peak_exercise_st_segment|3,thal|normal,thal|reversible_defect,chest_pain_type|2,chest_pain_type|3,chest_pain_type|4,num_major_vessels|1,num_major_vessels|2,num_major_vessels|3,resting_ekg_results|1,resting_ekg_results|2
0,0z64un,0,1,0,0,high,too_high,med,young,low,0,0,1,0,1,0,0,0,0,0,0,1
1,ryoo3j,0,0,0,0,low,high,low,young,high,1,0,1,0,0,1,0,0,0,0,0,0
2,yt1s1x,0,1,1,1,high,high,med,old,low,0,0,1,0,0,0,1,0,0,1,0,1
3,l2xjde,0,1,0,1,low,too_high,too_high,young,low,0,0,0,1,0,0,1,0,0,0,0,0
4,oyt4ek,0,1,0,0,high,med,too_high,old,high,0,1,0,1,0,0,0,0,0,0,0,1


In [0]:
all_dummied.nunique()

In [0]:
submis_df = pd.read_csv(test_vals)
submis_df.head()

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina
0,olalu7,2,reversible_defect,170,1,0,0,2,288,0.2,1,59,159,0
1,z9n6mx,1,normal,138,4,0,0,0,183,1.4,0,35,182,0
2,5k4413,2,reversible_defect,120,4,0,0,2,177,2.5,1,43,120,1
3,mrg7q5,1,normal,102,3,1,0,0,318,0.0,0,60,160,0
4,uki4do,2,normal,138,4,1,0,2,166,3.6,1,61,125,1


In [0]:
# create series
submis_serum = pd.cut(submis_df['serum_cholesterol_mg_per_dl'], bins = bins_serum, labels = 'low average high'.split(), include_lowest = True)
submis_hr = pd.cut(submis_df['max_heart_rate_achieved'], bins = bins_hr, labels = 'low med high too_high'.split(), include_lowest = True)
submis_bp = pd.cut(submis_df['resting_blood_pressure'], bins = bins_bp, labels = 'low med high too_high'.split(), include_lowest = True)
submis_age = pd.cut(submis_df['age'], bins = bins_age, labels = 'young old'.split(), include_lowest = True)
submis_depr = pd.cut(submis_df['oldpeak_eq_st_depression'], bins = bins_depr, labels = 'low mid high'.split(), include_lowest = True)

#rename series
submis_serum.rename('serum_bin', inplace = True)
submis_hr.rename('heart_rate_bin', inplace = True)
submis_bp.rename('blood_pressure_bin', inplace = True)
submis_age.rename('age_bin', inplace = True)
submis_depr.rename('depression_bin', inplace = True)

In [0]:
submis_df = pd.concat([submis_df, submis_serum, submis_hr, submis_bp, submis_age, submis_depr], axis = 1)


In [0]:
submis_df.isnull().sum()

patient_id                              0
slope_of_peak_exercise_st_segment       0
thal                                    0
resting_blood_pressure                  0
chest_pain_type                         0
num_major_vessels                       0
fasting_blood_sugar_gt_120_mg_per_dl    0
resting_ekg_results                     0
serum_cholesterol_mg_per_dl             0
oldpeak_eq_st_depression                0
sex                                     0
age                                     0
max_heart_rate_achieved                 0
exercise_induced_angina                 0
serum_bin                               0
heart_rate_bin                          3
blood_pressure_bin                      2
age_bin                                 0
depression_bin                          0
dtype: int64

In [0]:
submis_df['heart_rate_bin'].fillna('low', inplace = True)

In [0]:
submis_df[submis_df['heart_rate_bin'].isnull()]

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,serum_bin,heart_rate_bin,blood_pressure_bin,age_bin,depression_bin
71,9fkefu,2,reversible_defect,123,4,2,0,0,282,2.0,1,53,95,1,high,NaN,med,young,high
72,6uk6kl,2,reversible_defect,152,4,1,0,0,274,1.2,1,57,88,1,high,NaN,too_high,old,mid
82,r4hsar,2,normal,120,4,0,0,0,237,1.0,1,67,71,0,average,NaN,low,old,mid


In [0]:
submis_df.isnull().sum().sum()

0

In [0]:
submis_df.head(2)

,patient_id,slope_of_peak_exercise_st_segment,thal,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,serum_bin,heart_rate_bin,blood_pressure_bin,age_bin,depression_bin
0,olalu7,2,reversible_defect,170,1,0,0,2,288,0.2,1,59,159,0,high,high,too_high,old,mid
1,z9n6mx,1,normal,138,4,0,0,0,183,1.4,0,35,182,0,low,too_high,high,young,mid


In [0]:
submis_df = submis_df.drop(labels = to_drop, axis = 1)

In [0]:
submis_df_clean = pd.get_dummies(data = submis_df, columns = cats, prefix_sep = '|', drop_first = True)

In [0]:
for tr, te in zip(all_dummied.drop(labels = 'heart_disease_present', axis = 1).columns.tolist(),submis_df_clean.columns.tolist()):
    print(tr,'\t', te, '\n')

In [0]:
check_list = 0

for tr, te in zip(all_dummied.drop(labels = 'heart_disease_present', axis = 1).columns.tolist(),submis_df_clean.columns.tolist()):
    
    if tr != te:
        check_list += 1
    else:
        pass
    
check_list

0

In [0]:
to_dum = ['serum_bin',
 'heart_rate_bin',
 'blood_pressure_bin',
 'age_bin',
 'depression_bin']

all_dummied = pd.get_dummies(data= all_dummied, columns = to_dum, drop_first = True, prefix_sep = '|')
submis_df_clean = pd.get_dummies(data= submis_df_clean, columns = to_dum, drop_first = True, prefix_sep = '|')

In [0]:
# datasets are in same format, finally.

In [0]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
sm = SMOTE(random_state = 42)

In [0]:
x = all_dummied.drop(labels = ['patient_id', 'heart_disease_present'], axis = 1).values
y = all_dummied['heart_disease_present'].values

In [0]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split as tts, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense

from sklearn.metrics import log_loss, classification_report

In [0]:
x_train, x_test, y_train, y_test = tts(x_resampled, y_resampled, random_state = 42, test_size = .22)

print(x_train.shape, y_train.shape, '\n', 
     x_test.shape, y_test.shape)

(156, 26) (156,) 
 (44, 26) (44,)


In [0]:
gs.best_params_

{'lr__C': 0.00915,
 'lr__penalty': 'l2',
 'rfc__n_estimators': 3,
 'svm__C': 0.0085,
 'svm__kernel': 'poly',
 'xgb__gamma': 0.001,
 'xgb__learning_rate': 1,
 'xgb__n_estimators': 8}

In [0]:
voters = VotingClassifier(
    estimators = [
    ('svm', SVC(random_state = 42, probability = True)), 
    ('lr', LogisticRegression(random_state = 42)), 
    ('xgb', XGBClassifier(objective = 'binary:logistic')), 
    ('rfc', RandomForestClassifier())
    ], 
                         voting = 'soft')

params = {
    'lr__C':[.00915 ], 
         'lr__penalty':['l2'], 
         'svm__C':[.0085], 
         'svm__kernel':['poly'],
         'rfc__n_estimators':[3], 
         'xgb__n_estimators':[8], 
         'xgb__learning_rate':[1], 
         'xgb__gamma':[.001]
         }


gs = GridSearchCV(voters, 
                 param_grid = params, 
                 cv = 10, 
                 verbose = 3, 
                 n_jobs = -1)

gs.fit(x, y)

In [0]:
clf_opt = gs.best_estimator_

In [0]:
submis_np = submis_df_clean.drop(labels = 'patient_id', axis = 1).values

In [0]:
submission_df = pd.concat([submis_df_clean['patient_id'], pd.DataFrame(clf_opt.predict_proba(submis_np), columns = [0,'heart_disease_present']).drop(labels = 0, axis = 1)], axis = 1).set_index('patient_id')

submission_df.to_csv('submis2.csv')

In [0]:
from keras.optimizers import SGD

In [0]:
x_train, x_test, y_train, y_test = tts(x, y, random_state = 42, test_size = .22)

print(x_train.shape, y_train.shape, '\n', 
     x_test.shape, y_test.shape)

(140, 26) (140,) 
 (40, 26) (40,)


In [0]:
model = Sequential()

model.add(Dense(output_dim = 20 , input_dim = 26, activation = 'relu', init = 'uniform'))
model.add(Dense(output_dim = 8, activation = 'relu', init = 'uniform'))
model.add(Dense(output_dim = 16, activation = 'relu', init = 'uniform'))
# model.add(Dense(output_dim = 14, activation = 'relu'))
model.add(Dense(output_dim = 8, activation = 'relu', init = 'uniform'))
model.add(Dense(output_dim = 1, activation = 'sigmoid'))


# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'binary_crossentropy')

model.fit(x_train, y_train, epochs = 100, batch_size = 28)

print(log_loss(y_test, model.predict_proba(x_test)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=26, activation="relu", units=20, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=8, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=16, kernel_initializer="uniform")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=8, kernel_initializer="uniform")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `D

Epoch 1/100
140/140 [==============================] - 3s 22ms/step - loss: 0.6929 - acc: 0.5286
Epoch 2/100
140/140 [==============================] - 0s 229us/step - loss: 0.6919 - acc: 0.5929
Epoch 3/100
140/140 [==============================] - 0s 224us/step - loss: 0.6911 - acc: 0.5929
Epoch 4/100
140/140 [==============================] - 0s 221us/step - loss: 0.6901 - acc: 0.5929
Epoch 5/100
140/140 [==============================] - 0s 230us/step - loss: 0.6894 - acc: 0.5929
Epoch 6/100
140/140 [==============================] - 0s 232us/step - loss: 0.6887 - acc: 0.5929
Epoch 7/100
140/140 [==============================] - 0s 271us/step - loss: 0.6878 - acc: 0.5929
Epoch 8/100
140/140 [==============================] - 0s 257us/step - loss: 0.6868 - acc: 0.5929
Epoch 9/100
140/140 [==============================] - 0s 213us/step - loss: 0.6856 - acc: 0.5929
Epoch 10/100
140/140 [==============================] - 0s 250us/step - loss: 0.6843 - acc: 0.5929
Epoch 11/100
140/140

In [0]:
print(log_loss(y_test, model.predict_proba(x_test)))


0.46377561772605985


In [0]:
submis3 = pd.concat([submis_df_clean['patient_id'] , pd.DataFrame(model.predict_proba(submis_np))], axis = 1).set_index('patient_id')

In [0]:
submis3.rename({0:'heart_disease_present'}, axis = 1, inplace = True)

submis3.to_csv('submis_dnn2.csv')